# Prep

In [52]:
import musicbrainzngs
import pandas as pd

from ratelimit import limits, sleep_and_retry
import traceback

# Authentication

In [53]:
musicbrainzngs.set_useragent(app="Opus Metadata Fetcher",version = "1.0")

# Add Musicbrainz metadata

## Utility Functions

In [54]:
def mb_recording_template(input_isrc = ''):
    # default recording_id in the format of music brainz recording id
    default_recording_id = 'zzzzzzzz-zzzz-zzzz-zzzz-zzzzzzzzzzzz'
    default_mb_recording_title = ''
    default_mb_recording_length = '0'
    
    data = {
    'mb_recording_id': [default_recording_id],
    'mb_recording_title': [default_mb_recording_title],
    'mb_recording_length': [default_mb_recording_length],
    'isrc_opus': [input_isrc]
}
    default_recording_df = pd.DataFrame(data)
    
    return default_recording_df

def compare_dataframes(df1, df2):
    # Check if column names are equal
    if list(df1.columns) != list(df2.columns):
        print("Column names are not equal")
        return False
    
    # Check if column types are equal
    if not df1.dtypes.equals(df2.dtypes):
        print("Column types are not equal")
        return False
    
    # print("Column names and types are equal")
    return True
def create_mb_globals():
    # dictionary of globals
    
    # default df
    mb_recording_df_dflt = mb_recording_template()
    
    found_recording_dfs = []
    missing_recording_dfs = []
    mb_release_dfs = []
    mb_label_dfs = []
    error_log = []

    mb_global_dict = {'recording_df_default':mb_recording_df_dflt,
                      'found_recording_dfs':found_recording_dfs,
                      'missing_recording_dfs':missing_recording_dfs,
                      'release_dfs':mb_release_dfs,
                      'label_dfs':mb_label_dfs,
                      'error_log':error_log}
    return mb_global_dict




def format_recording(mb_recording,input_isrc):
    # formats music brains recording object
    recording_df = pd.DataFrame(mb_recording['isrc']['recording-list'])
    recording_df['isrc_opus'] = input_isrc
    recording_df = recording_df.rename(columns={'id':'mb_recording_id',
                                                    'title':'mb_recording_title',
                                                    'length':'mb_recording_length'})
    return recording_df

### Release

In [55]:
def parse_release_text(text):
    """retrieve language from text-representation field of release dataframe
    ex: {'language': 'eng', 'script': 'Latn'} --> 'eng' """
    try: 
        # format string as a dictionary and extract value of 'language' key
        language = dict(text)['language']
    except Exception as e:
        language = None
        print(e)
    finally:
        return language
def parse_release_date(date_str):
    if len(date_str) == 4:
        return date_str + '-01-01'  # Assuming it's the start of the year
    elif len(date_str) == 7:
        return date_str + '-01'
    else:
        return date_str
    
def flag_complete_date(date_str):
    try:
        pd.to_datetime(date_str,format='%Y-%m-%d')
        return '1'
    except ValueError:
        return '0'
def format_mb_release(mb_release,recording_id):
    release_df = pd.DataFrame(mb_release['release-list'])
    # print(mb_release.columns)
    # get release language from text-representation field
    release_df['mb_release_language'] = release_df['text-representation'].apply(parse_release_text)
    # flag date format
    release_df['mb_complete_date_flag'] = release_df['date'].apply(flag_complete_date)
    # format date
    release_df['mb_date'] = release_df['date'].apply(parse_release_date)
    
    # add recording_id
    release_df['mb_recording_id'] = recording_id
    # rename columns
    release_df = release_df.rename(columns = {'id':'mb_release_id',
                                              'title':'mb_release_title',
                                              'status':'mb_release_status',
                                              'country':'mb_release_country',
                                              'quality':'mb_release_quality'})
    
    release_df['recording_id'] = recording_id
    # select columns
    release_df = release_df[['mb_release_id',
                             'mb_release_title',
                             'mb_release_status',
                             'mb_release_country',
                             'mb_release_quality',
                             'mb_release_language',
                             'mb_complete_date_flag',
                             'mb_date',
                             'mb_recording_id']]
    return release_df
    # print('done')


In [56]:
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
# define function
def build_release_df(input_recording_id,mb_global_dict):
    # if HTTPError, handle with default df
    try:
        # look up recording using isrc. if the isrc is not found, ResponseError will be thrown
        releases = musicbrainzngs.browse_releases(recording=input_recording_id)
        release_df = format_mb_release(releases,recording_id=input_recording_id)
        mb_global_dict['release_dfs'].append(release_df)
    except musicbrainzngs.musicbrainz.ResponseError as e:
        print('recording not found in musicbrainz')

    finally:
        return mb_global_dict

## Build recording df

In [57]:
# plan
mb_global_dict = create_mb_globals()
# isrc --> recording --> release --> label
input_isrcs = ['USSM10600677',
               'USSM106006771',
               'USSM106006772',
               'USSM106006773',
               'USSM106006774',
               'USSM106006775']
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
def build_recording_df(input_isrc,mb_global_dict):
    # if HTTPError, handle with default df
    try:
        # look up recording using isrc. if the isrc is not found, ResponseError will be thrown
        recordings = musicbrainzngs.get_recordings_by_isrc(input_isrc, includes=[], release_status=[], release_type=[])
        # if isrc is found, then format the recording information
        recording_df = format_recording(recordings,input_isrc = input_isrc)
        assert compare_dataframes(recording_df,mb_global_dict['recording_df_default'])
        # add to list of found recordings
        mb_global_dict['found_recording_dfs'].append(recording_df)
    except musicbrainzngs.musicbrainz.ResponseError as e:
        print('isrc not found in musicbrainz')
        # create missing recording info for isrc
        recording_df = mb_recording_template(input_isrc=input_isrc)
        # ensure proper format 
        assert compare_dataframes(recording_df,mb_global_dict['recording_df_default'])
        # add to missing recording dfs
        mb_global_dict['missing_recording_dfs'].append(recording_df)
    
    except Exception as e:
        print(e)
        error_log = {'process':'recording_df',
                     'error_type':'fatal',
                     'error':e}
        mb_global_dict['error_log'].append(error_log)
        # raise e
        
    
    finally:
        return mb_global_dict

# dim_song_detail --> stg_mb_recording_detail (key: isrc, mb_recording_id)
# stg_mb_release_detail (key: recording/release)
# stg_mb_label_detail (key: label/release)

In [58]:
# Call the function ensuring one call per second
for isrc in input_isrcs:
    try:
        mb_global_dict = build_recording_df(input_isrc=isrc,mb_global_dict = mb_global_dict)
    except Exception as e:
        raise e

isrc not found in musicbrainz
isrc not found in musicbrainz
isrc not found in musicbrainz
isrc not found in musicbrainz
isrc not found in musicbrainz


## Build release df

In [59]:
for df in mb_global_dict['found_recording_dfs']:
    for recording_id in df['mb_recording_id']:
        build_release_df(recording_id,mb_global_dict)

In [62]:
mb_global_dict['release_dfs'][1]

,mb_release_id,mb_release_title,mb_release_status,mb_release_country,mb_release_quality,mb_release_language,mb_complete_date_flag,mb_date,mb_recording_id
0,27af023b-7967-4cc4-8ef5-7edccada8219,Die ultimative Chart Show: Die erfolgreichsten...,Official,DE,normal,mul,1,2009-11-13,53056bc7-d60e-4f68-b420-11894097fc09
1,5215b593-5741-47b3-bd4e-d5a3d473068f,School Disco Anthems,Official,AU,normal,eng,1,2010-05-25,53056bc7-d60e-4f68-b420-11894097fc09
2,ee5ac04a-143f-4371-b90f-5c8e0b5e5751,Zomer Top 100: Joe FM,Official,BE,normal,eng,1,2012-06-22,53056bc7-d60e-4f68-b420-11894097fc09


# Scratch Work